In [25]:
import numpy as np, pandas as pd
from tqdm import tqdm_notebook
import json

from pathlib import Path
import os
import lightgbm as lgbm
import csv
import operator

## Prepare the data

In [26]:
def load_x_data(file):

    with open(f'{file}', 'r', encoding='utf-8') as f:
        input_lines = f.read().split('\n')

    data = {
        "title": [],
        "description": [],
        "recent_posts": [],
    }

    for line in input_lines:
        try:
            line = json.loads(line)
        except:
            pass

        if line != '':
            try:
                data['title'].append(line['title'])
                data['description'].append(line['description'])
                data['recent_posts'].append('\n'.join(line['recent_posts']))
            except:
                print('Parse error')

    data = pd.DataFrame(data)
    return data

def load_y_data(file):

    with open(f'{file}', 'r', encoding='utf-8') as f:
        input_lines = f.read().split('\n')

    data = {
        #'lang_code': [],
        'label': []  
    }

    for line in input_lines:
        try:
            line = json.loads(line)
        except:
            pass

        if line != '':
            try:
                #data['lang_code'].append(line['lang_code'])
                data['label'].append('<SEP>'.join(list(line['category'].keys())))
            except:
                print('Parse error')
    data = pd.DataFrame(data)
    return data

In [27]:
x_data = pd.DataFrame(columns = ['title', 'description', 'recent_posts'])
y_data = pd.DataFrame(columns = ['label'])
PATH = '../data/category_en/'
x_data = pd.concat([x_data, load_x_data(str(PATH + "input.txt"))]).reset_index(drop=True)
y_data = pd.concat([y_data, load_y_data(str(PATH + "output.txt"))]).reset_index(drop=True)
    
x_data['recent_posts'] = x_data['recent_posts'].apply(lambda x: x.replace('\u200b', ''))
data = x_data
data['label'] = y_data

In [28]:
data.head()

,title,description,recent_posts,label
0,BELAIRE🥃❤️,Don't try🔪❤️,colorado 🤣\nnaa wop3 asem o\n🤣🤣🤣🤣🤣🤣🤣\nlol okay...,Bets & Gambling<SEP>Cars & Other Vehicles<SEP>...
1,SK365 ROLLOVERS & INPLAYS,Daily 2-20 odds. Rollover challenge. Small odd...,Risky inplay 💚🥶\nInplay 4+ Odds \n\nVery Risky...,Bets & Gambling<SEP>Investments<SEP>Politics &...
2,Weirdo thoughts™️😏,"Weirdo gang 😪: weirdo quotes, pic type shit th...",@weirdo_thoughts\n@weirdo_thoughts\nLet's just...,Directories of Channels & Bots<SEP>Education<S...
3,Social 👟 Media🍷Trends 🔞,,I'll make one😅\n💡 @social_media_trends\n🎶 Trac...,Celebrities & Lifestyle<SEP>Curious Facts<SEP>...
4,The Premier League,This is a channel to get information on Premie...,8 - Only versus Arsenal (11) has Jamie Vardy s...,Sports


In [29]:
mapper = {'Art & Design': '__label__0',
 'Bets & Gambling': '__label__1',
 'Books': '__label__2',
 'Business & Entrepreneurship': '__label__3',
 'Cars & Other Vehicles': '__label__4',
 'Celebrities & Lifestyle': '__label__5',
 'Cryptocurrencies': '__label__6',
 'Culture & Events': '__label__7',
 'Curious Facts': '__label__8',
 'Directories of Channels & Bots': '__label__9',
 'Economy & Finance': '__label__10',
 'Education': '__label__11',
 'Erotic Content': '__label__12',
 'Fashion & Beauty': '__label__13',
 'Fitness': '__label__14',
 'Food & Cooking': '__label__15',
 'Foreign Languages': '__label__16',
 'Health & Medicine': '__label__17',
 'History': '__label__18',
 'Hobbies & Activities': '__label__19',
 'Home & Architecture': '__label__20',
 'Humor & Memes': '__label__21',
 'Investments': '__label__22',
 'Job Listings': '__label__23',
 'Kids & Parenting': '__label__24',
 'Marketing & PR': '__label__25',
 'Motivation & Self-Development': '__label__26',
 'Movies': '__label__27',
 'Music': '__label__28',
 'Offers & Promotions': '__label__29',
 'Pets': '__label__30',
 'Politics & Incidents': '__label__31',
 'Psychology & Relationships': '__label__32',
 'Real Estate': '__label__33',
 'Recreation & Entertainment': '__label__34',
 'Religion & Spirituality': '__label__35',
 'Science': '__label__36',
 'Sports': '__label__37',
 'Technology & Internet': '__label__38',
 'Travel & Tourism': '__label__39',
 'Video Games': '__label__40',
 'Other': '__label__41'}

reverse_mapper = {v: k for k, v in mapper.items()}

In [30]:
y_data = pd.DataFrame(columns=mapper.keys())
y_data

,Art & Design,Bets & Gambling,Books,Business & Entrepreneurship,Cars & Other Vehicles,Celebrities & Lifestyle,Cryptocurrencies,Culture & Events,Curious Facts,Directories of Channels & Bots,...,Psychology & Relationships,Real Estate,Recreation & Entertainment,Religion & Spirituality,Science,Sports,Technology & Internet,Travel & Tourism,Video Games,Other


In [31]:
labels_list = pd.Series(data.label.apply(lambda x: x.split("<SEP>")))
y_data_preprocesse = pd.get_dummies(labels_list.apply(pd.Series).stack(), columns=mapper.keys()).sum(level=0)
y_data_preprocesse.head()

,Art & Design,Bets & Gambling,Books,Business & Entrepreneurship,Cars & Other Vehicles,Celebrities & Lifestyle,Cryptocurrencies,Culture & Events,Curious Facts,Directories of Channels & Bots,...,Pets,Politics & Incidents,Psychology & Relationships,Real Estate,Religion & Spirituality,Science,Sports,Technology & Internet,Travel & Tourism,Video Games
0,0,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [32]:
print(y_data_preprocesse.iloc[0])

Art & Design                      0
Bets & Gambling                   1
Books                             0
Business & Entrepreneurship       0
Cars & Other Vehicles             1
Celebrities & Lifestyle           0
Cryptocurrencies                  0
Culture & Events                  0
Curious Facts                     1
Directories of Channels & Bots    0
Economy & Finance                 0
Education                         1
Erotic Content                    0
Fashion & Beauty                  0
Fitness                           0
Food & Cooking                    1
Foreign Language Learning         0
Health & Medicine                 0
History                           0
Hobbies & Activities              0
Home & Architecture               0
Humor & Memes                     0
Investments                       0
Job Listings                      0
Kids & Parenting                  0
Marketing & PR                    0
Motivation & Self-development     0
Movies                      

In [33]:
data.label.iloc[0]

'Bets & Gambling<SEP>Cars & Other Vehicles<SEP>Curious Facts<SEP>Education<SEP>Food & Cooking<SEP>Movies<SEP>Politics & Incidents<SEP>Sports'

In [34]:
data = data.drop("label", axis=1)
data.head()

,title,description,recent_posts
0,BELAIRE🥃❤️,Don't try🔪❤️,colorado 🤣\nnaa wop3 asem o\n🤣🤣🤣🤣🤣🤣🤣\nlol okay...
1,SK365 ROLLOVERS & INPLAYS,Daily 2-20 odds. Rollover challenge. Small odd...,Risky inplay 💚🥶\nInplay 4+ Odds \n\nVery Risky...
2,Weirdo thoughts™️😏,"Weirdo gang 😪: weirdo quotes, pic type shit th...",@weirdo_thoughts\n@weirdo_thoughts\nLet's just...
3,Social 👟 Media🍷Trends 🔞,,I'll make one😅\n💡 @social_media_trends\n🎶 Trac...
4,The Premier League,This is a channel to get information on Premie...,8 - Only versus Arsenal (11) has Jamie Vardy s...


In [35]:
#data = pd.concat([data, y_data_preprocesse], axis=1, join="inner")
#data.head()

In [36]:
# Remove emojis
import re


def deEmojify(text):    
    regex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"                    
                      "]+", re.UNICODE)
    return regex_pattern.sub(r'',text)

data['recent_posts'] = data['recent_posts'].apply(deEmojify)
data['title'] = data['title'].apply(deEmojify)
data['description'] = data['description'].apply(deEmojify)

In [37]:
data['recent_posts'] = data['recent_posts'].apply(lambda x: x.lower())
data['title'] = data['title'].apply(lambda x: x.lower())
data['description'] = data['description'].apply(lambda x: x.lower())

In [38]:
# Remove adds from all posts

ALL_POSTS = []
for v in tqdm_notebook(data['recent_posts'].apply(lambda x: x.split('\n'))):
    ALL_POSTS.extend(v)
post_counts = pd.Series(ALL_POSTS).value_counts().sort_values(ascending = False)


def filter_posts(posts, threshold = 5):
    posts = posts.split('\n')
    filtered_posts = []
    for post in posts:
        if post_counts[post] < threshold:
            filtered_posts.append(post)
    return '\n'.join(filtered_posts)

for i in range(data.shape[0]):
    data.iloc[i, 2] = filter_posts(data.iloc[i, 2])

<ipython-input-38-be64877c48f7>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for v in tqdm_notebook(data['recent_posts'].apply(lambda x: x.split('\n'))):


In [39]:
def removeEmail(text):
    pattern = re.compile("((\w+)(\.|_)?(\w*)@(\w+)(\.(\w+))+)")
    return pattern.sub(r'', text)

data['recent_posts'] = data['recent_posts'].apply(removeEmail)
data['title'] = data['title'].apply(removeEmail)
data['description'] = data['description'].apply(removeEmail)

In [40]:
def removeUsername(text):
    pattern = re.compile("(@(\w+))")
    return pattern.sub(r'', text)

data['recent_posts'] = data['recent_posts'].apply(removeUsername)
data['title'] = data['title'].apply(removeUsername)
data['description'] = data['description'].apply(removeUsername)

In [41]:
def removeLinks(text):
    pattern = re.compile("(https?://[^ ]+)")
    return pattern.sub(r'', text)

data['recent_posts'] = data['recent_posts'].apply(removeLinks)
data['title'] = data['title'].apply(removeLinks)
data['description'] = data['description'].apply(removeLinks)

In [42]:
from tqdm import tqdm
tqdm.pandas()

/home/sokol/.local/lib/python3.8/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [43]:
from sklearn.model_selection import train_test_split


data['preprocessed_text'] = data.title.map(str) + data.description.map(str) + ' ' + data.recent_posts.map(str)
data = data.drop(['title', 'description', 'recent_posts'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(data, y_data_preprocesse, shuffle=True, train_size=0.7)

In [44]:
print(X_train.shape, X_test.shape)

(10342, 1) (4433, 1)


In [45]:
 from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [46]:
X_train.shape

(10342, 1)

## Tokenize and vectorize text

In [47]:
tfidf_params = {'max_features':3000, 'use_idf':True}

tfidf = TfidfVectorizer(**tfidf_params)
tfidf_trained = tfidf.fit(X_train.preprocessed_text)
x_train_as_array = tfidf_trained.transform(X_train.preprocessed_text).toarray()

x_test_as_array = tfidf_trained.transform(X_test.preprocessed_text).toarray()

print(x_train_as_array.shape)

(10342, 3000)


## Save TfIdf vectorizer

In [48]:

tfidf_vocabulary_df = pd.Series(tfidf_trained.vocabulary_)
tfidf_vocabulary_df.to_csv('../models/en/tfidf/tfidf_vocabulary.csv', header=False)

tfidf_idf_df = pd.Series(tfidf_trained.idf_)
tfidf_idf_df.to_csv('../models/en/tfidf/tfidf_idf.csv', header=False)

tfidf_params_df = pd.Series(tfidf_params)
tfidf_params_df.to_csv('../models/en/tfidf/tfidf_params.csv', header=False)


In [49]:
test_preprocess_data = pd.DataFrame(X_train)
test_preprocess_data.to_csv('../data/category_en/test_preprocessed_data.csv', index=False, header=False)
test_preprocess_data.head()

,preprocessed_text
12167,dimpy super surplus storecontact - 989127584...
2789,gg’s marketplace welcome to gg’s marketplace !...
2770,reentry h1 (zzl)reentry h1 signal (zzl) zzl se...
9296,online shopping loot deal offerfor any queries...
13623,"rp.in this world,\neveryone has a to ,\nso, j..."


## Evaluate model

In [50]:
from sklearn.multiclass import OneVsRestClassifier

In [51]:
params = { 
	# ... 
    'objective': 'binary',
    'boosting_type': 'gbdt', 
    'metric': 'logistic',
    "learning_rate" : 0.01,
    'task': 'train'
} 

In [52]:
model = OneVsRestClassifier(lgbm.LGBMClassifier(**params), n_jobs=3)

In [53]:
model = model.fit(x_train_as_array, y_train.values.tolist()) 

In [54]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score, roc_auc_score
y_pred = model.predict(x_test_as_array)
print('Model score ', model.score(x_test_as_array, y_test))
print('Accuracy ', accuracy_score(y_test, y_pred))
print('F1 ', f1_score(y_test, y_pred.astype(float), average='micro'))
print('Precision ', precision_score(y_test, y_pred.astype(float), average='micro'))
print('Recall ', recall_score(y_test, y_pred.astype(float), average='micro'))
print(y_pred[0])
print(y_test.iloc[0].values)

Model score  0.11594856756147079
Accuracy  0.11594856756147079
F1  0.4265780245470485
Precision  0.8764635244671269
Recall  0.28188664671236846
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
 0 1 0 0]


## Save lightmgbm models

In [55]:
for i in range(len(model.estimators_)):
    model.estimators_[i].booster_.save_model('../models/en/lightgbm_model_' + str(i) + '.txt')